In [1]:
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
import contractions
import spacy
import nltk
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from nltk.corpus import wordnet

import json
from dataclasses import dataclass
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer, util
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline, AutoModel
from transformers import BertTokenizer, BertModel
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# load require package

import langchain
from langchain.document_loaders import HuggingFaceDatasetLoader
#from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA

from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from transformers import AutoModelForCausalLM, FalconForQuestionAnswering, FalconConfig
import transformers
import torch

from langchain.vectorstores import VectorStore
import pprint
import random
import string

from fuzzywuzzy import fuzz

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

# Ensure Spacy model is installed
try:
    nlp = spacy.load('en_core_web_sm')
except OSError:
    from spacy.cli import download
    download('en_core_web_sm')
    nlp = spacy.load('en_core_web_sm')

c:\Users\SimWen\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\SimWen\AppData\Local\Programs\Python\Python39\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\SimWen\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\SimWen\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\SimWen\AppData\Roaming\nltk_data

#### Text Preprocessing

In [2]:
# Define a function to convert mixed case to lower case
def convert_mixed_case_to_lower(x):
    if isinstance(x, str):
        lines = x.split('\n')
        new_lines = []
        for line in lines:
            words = line.split()
            new_words = []
            for word in words:
                if word.isupper():
                    new_words.append(word)
                else:
                    new_words.append(word.lower())
            new_lines.append(' '.join(new_words))
        return ' '.join(new_lines)
    return x

# Function to remove special character or ID patterns
# def remove_special_ids(text, pattern=r'\b\w*\d\w*\b'):
#     if isinstance(text, str):
#         return re.sub(pattern, ' ', text)
#     return text

def remove_punctuation(text):
        text = re.sub(r'/', 'or', text)
        text = re.sub(r'&', 'and', text)
        text = re.sub(r'[^\w\s]', ' ', text)
        # text = re.sub(r'(?<!\d)\.(?!\d)|[^\w\s.]', ' ', text)
        return text

def tokenize(text):
    return word_tokenize(text)

def remove_stopwords(tokens):
    stop_words = set(stopwords.words('english'))
    return [word for word in tokens if word not in stop_words]

def stemming(tokens):
    stemmer = PorterStemmer()
    return [stemmer.stem(word) for word in tokens]

def lemmatization(tokens):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(word) for word in tokens]

def expand_contractions(text):
    return contractions.fix(text)

def remove_numbers(text):
    return re.sub(r'\d+', ' ', text)

def remove_extra_whitespace(text):
    return ' '.join(text.split())

# def remove_special_chars(df, columns):
#     for col in columns:
#         df[col] = df[col].apply(lambda x: re.sub(r'[^\w\s]', '', x))
#     return df

def preprocess(text):
    text = convert_mixed_case_to_lower(text)
    # text = remove_special_ids(text)
    text = expand_contractions(text)
    text = remove_punctuation(text)
    # text = remove_numbers(text)
    # text = remove_special_chars(text)
    text = remove_extra_whitespace(text)
    tokens = tokenize(text)
    tokens = remove_stopwords(tokens)
    tokens = lemmatization(tokens)  # or use stemming(tokens)
    return ' '.join(tokens)

### Clean Query For Embedding Searching, Raw Query for LLM Summarization

In [3]:
# Define your query
#What is the detail of wrong wafer barcode scanner, and what are the action should be taken when facing this failure?
query = "i want a scenario that can prevent the failure"
preprocess_query = preprocess(query)
print(preprocess_query)

want scenario prevent failure


### Load Vectore Store

In [4]:
# load vectorstore.pkl file from local
vectorstore_file = "multi-qa-MiniLM-L6-cos-v1.pkl"

with open(vectorstore_file, "rb") as f:
    global vectorstore
    db: VectorStore = pickle.load(f)

In [5]:

# Define the path to the pre-trained model you want to use
modelPath = "sentence-transformers/multi-qa-MiniLM-L6-cos-v1"

# Create a dictionary with model configuration options, specifying to use the CPU for computations
model_kwargs = {'device':'cpu'}

# Create a dictionary with encoding options, specifically setting 'normalize_embeddings' to False
encode_kwargs = {'normalize_embeddings': True}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs, # Pass the encoding options
)

C:\Users\SimWen\AppData\Local\Temp\ipykernel_14988\649506487.py:11: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


In [6]:
# Define the query and expand it
def expand_query(query):
    synonyms = set()
    for word in query.split():
        for syn in wordnet.synsets(word):
            for lemma in syn.lemmas():
                synonyms.add(lemma.name())
    return query + " " + " ".join(synonyms)

def word_match_score(query, document):
    query_words = set(query.lower().split())
    doc_words = set(document.lower().split())
    match_count = len(query_words.intersection(doc_words))
    return match_count / len(query_words)

In [7]:
def tf_idf_Filtering(db, n_doc, query, k):
    preprocess_query = preprocess(query)
    # Extract document contents
    all_docs = db.similarity_search(preprocess_query, k = n_doc)
    doc_texts = [doc.page_content for doc in all_docs]

    # Initialize the TF-IDF vectorizer
    tfidf_vectorizer = TfidfVectorizer()

    # Calculate TF-IDF matrix for all documents
    tfidf_matrix = tfidf_vectorizer.fit_transform(doc_texts)

    expanded_query = expand_query(preprocess_query)

    # Calculate TF-IDF scores for the query
    query_tfidf_vector = tfidf_vectorizer.transform([expanded_query])
    
    # Calculate cosine similarity between query TF-IDF vector and candidate TF-IDF vectors
    cosine_similarities = cosine_similarity(tfidf_matrix, query_tfidf_vector).flatten()

    # Calculate fuzzy matching scores
    fuzzy_scores = [fuzz.partial_ratio(expanded_query, doc.page_content) / 100.0 for doc in all_docs]

    # Calculate word matching scores
    word_match_scores = [word_match_score(expanded_query, doc.page_content) for doc in all_docs]

    # Combine the scores with additional word matching
    combined_scores = [
        ((0.5 * cos_sim + 0.8 * fuzz_score + 0.8 * word_match) / (0.5 + 0.8 + 0.8)) 
        for cos_sim, fuzz_score, word_match in zip(cosine_similarities, fuzzy_scores, word_match_scores)
    ]

    # Sort candidates based on TF-IDF cosine similarity
    sorted_indices = sorted(range(len(combined_scores)), key=lambda i: combined_scores[i], reverse=True)
    filtered_docs = [all_docs[i] for i in sorted_indices]
    filtered_scores = [combined_scores[i] for i in sorted_indices]

    return filtered_docs[:k], filtered_scores[:k], int(n_doc * 0.5)

def filterEmbedding_db(filtered_docs, n_doc):
    filter_db = FAISS.from_documents(filtered_docs[:n_doc], embeddings)
    return filter_db, int(n_doc * 0.5)

In [8]:
filtered_docs, filtered_scores, n_docs = tf_idf_Filtering(db, 50, query, 2)
# db = FAISS.from_documents(filtered_docs, embeddings)
# filtered_docs = db.similarity_search(preprocess_query, k = 10)
retrieve_docs = []
i=1
for doc, score in zip(filtered_docs, filtered_scores):
    print("Document Metadata:")
    print(f"Similarity Score: {score}")
    print("\n")
    temp = []
    
    for key, value in doc.metadata.items():
        temp.append(f"{key}: {value} ")
        print(i,")",f"{key}: {value}")

        print("\n")
       
    retrieve_docs.append(' '.join(temp))
    print("\n----------------------------\n\n")
    i+=1 

Document Metadata:
Similarity Score: 0.16965472960071526


1 ) question: Failure cause of the work element : 
50.13.1.7 Destroy the wire for reject unit
[Method_Z8K00014817]




1 ) Function of the process step and product characteristic: 50.0 Bonding station


1 ) Failure effect FE to next higher level element and/or end user: Customer: Electrical/Functionality/Reliability Failure
[Z8K00014817]
IFX: Yield loss
[Z8K00014817]


1 ) Failure mode FM of the focus element: Stray Wire


1 ) Current preventive control PC for FC: i) No destroy wire is require, use reject slip to identify reject unit slot, ink dot on reject die , and punch the lead during Dias inspection (P)

iii) New design of Stray wire vacuum suction bin (P)


1 ) Current detection control DC for FC or FM: ii) Visual Inspection - During 2hourly inspection / 3rd Optical inspection / Every change/replacement capillary / Every change device / Safe Release (D)


1 ) subject: Unit Process FMEA for QFP+ Wire Bond Copper Wire


1 )

In [9]:
retrieve_docs

['question: Failure cause of the work element : \n50.13.1.7 Destroy the wire for reject unit\n[Method_Z8K00014817]\n\n  Function of the process step and product characteristic: 50.0 Bonding station  Failure effect FE to next higher level element and/or end user: Customer: Electrical/Functionality/Reliability Failure\n[Z8K00014817]\nIFX: Yield loss\n[Z8K00014817]  Failure mode FM of the focus element: Stray Wire  Current preventive control PC for FC: i) No destroy wire is require, use reject slip to identify reject unit slot, ink dot on reject die , and punch the lead during Dias inspection (P)\n\niii) New design of Stray wire vacuum suction bin (P)  Current detection control DC for FC or FM: ii) Visual Inspection - During 2hourly inspection / 3rd Optical inspection / Every change/replacement capillary / Every change device / Safe Release (D)  subject: Unit Process FMEA for QFP+ Wire Bond Copper Wire  package: QFP+  location: Manufacturing-Melaka  fmeaIid: Z8K00014817  fmeaidx_id: Z8K00

In [10]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
import contractions
from nltk.corpus import wordnet

# 下载必要的NLTK数据
import nltk


# 定义函数将混合大小写转换为小写
def convert_mixed_case_to_lower(text):
    if isinstance(text, str):
        return text.lower()
    return text

# 去除标点符号
def remove_punctuation(text):
    text = re.sub(r'/', ' or ', text)
    text = re.sub(r'&amp;', ' and ', text)
    text = re.sub(r'[^\w\s]', ' ', text)
    return text

# 扩展缩写
def expand_contractions(text):
    return contractions.fix(text)

# 去除停用词
def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text)
    filtered_text = [word for word in word_tokens if word.lower() not in stop_words]
    return ' '.join(filtered_text)

# 词干提取
def stem_words(text):
    stemmer = PorterStemmer()
    word_tokens = word_tokenize(text)
    stemmed_text = [stemmer.stem(word) for word in word_tokens]
    return ' '.join(stemmed_text)

# 词形还原
def lemmatize_words(text):
    lemmatizer = WordNetLemmatizer()
    word_tokens = word_tokenize(text)
    lemmatized_text = [lemmatizer.lemmatize(word, get_wordnet_pos(word)) for word in word_tokens]
    return ' '.join(lemmatized_text)

# 获取词性
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ, "N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

# 综合预处理函数
def preprocess_input(text):
    text = convert_mixed_case_to_lower(text)
    text = expand_contractions(text)
    text = remove_punctuation(text)
    text = remove_stopwords(text)
    text = lemmatize_words(text)
    return text

# 示例使用
user_input = "I'm looking for the best AI tools & techniques in 2024!"
processed_input = preprocess_input(user_input)
print(processed_input)

look best ai tool technique 2024
